In [220]:
using LinearAlgebra
using Statistics
using Distributions
using SpecialFunctions
using Combinatorics
using Plots
using ProximalOperators

In [308]:
include("./graph_decomposition.jl")
include("./normalising_constant_mc.jl")
include("./normalising_constant_laplace.jl")
include("./helperfunctions.jl")
include("particle_MCMC.jl")

logIno (generic function with 1 method)

# Generate precision matrices and distributions

In [3]:
n_nodes = 10
mean = zeros(n_nodes)
#covariance matrix - precision matrix is its inverse which is also identity
Id = Matrix(1.0I, n_nodes, n_nodes)

10×10 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [140]:
P1 = copy(Id)
for i in 1:(n_nodes - 1)
    P1[i, i + 1] = 0.5
    P1[i + 1, i] = 0.5
end
for i in 1:(n_nodes - 2)
    P1[i, i + 2] = 0.4
    P1[i + 2, i] = 0.4
end

In [145]:
distr1 = MvNormal(zeros(n_nodes), P1)

FullNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.5 … 0.0 0.0; 0.5 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.5; 0.0 0.0 … 0.5 1.0]
)


In [157]:
dist1 = MvNormalCanon(P1)

ZeroMeanFullNormalCanon(
μ: 10-element Zeros{Float64}
h: 10-element Zeros{Float64}
J: [1.0 0.5 … 0.0 0.0; 0.5 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.5; 0.0 0.0 … 0.5 1.0]
)


In [158]:
rand(dist1)

10-element Array{Float64,1}:
  0.4224870639363323
  0.044758002463508306
  0.2586279536271129
 -1.4392653698721827
  2.181106692597452
 -1.3185097913234998
 -2.8171644808100087
  0.5947312312947794
  1.9825795526367438
 -1.0160640888779264

In [5]:
#first and second off-diagonal elements

function diagonal_pairs(n_nodes)
    N_edges = (n_nodes - 1) + (n_nodes - 2)
    res = [fill(1, 1) for i in 1:N_edges]
    counter = 1
    for i in 1:(n_nodes - 1)
        res[counter] = [i, i+1]
        counter += 1
        #push!(res, [i+1, i])
    end
    for i in 1:(n_nodes - 2)
        res[counter] = [i, i+2]
        counter += 1
        #push!(res, [i+2, i])
    end
    return res
end

diagonal_pairs (generic function with 1 method)

In [6]:
#other elements 3rd 4th off-diagonal etc

function offdiagonal_pairs(n_nodes)
    
    N_edges = Int64((n_nodes-2)*(n_nodes-3)/2)

    res = [fill(1, 1) for i in 1:N_edges]

      sum = 0

      for i in 1:(n_nodes-1)

        for j in (i+3):n_nodes

          res[sum+j-(i+2)] = [i,j]

        end

        sum = sum + n_nodes - (i+2)

      end

    return res
end



offdiagonal_pairs (generic function with 1 method)

In [217]:
#to get P2 select 5 edges at random (non-diagonal) and set them equal to 0
#then add 5 edges (set the value equal to 0.5) not present in P1

function mutate(P)
    
    Ppr = copy(P)
    
    ndelete = Int64(ceil(n_nodes / 2))
    
    nadd = Int64(ceil(n_nodes / 2))
    
    diagonal = diagonal_pairs(n_nodes)
    offdiagonal = offdiagonal_pairs(n_nodes)
    
    delete_edges = sample(diagonal, ndelete, replace=false)
    
    add_edges = sample(offdiagonal, nadd, replace=false)
    
    for i in 1:ndelete
        
        edge = delete_edges[i]
        fst = edge[1]
        snd = edge[2]
        Ppr[fst, snd] = 0
        Ppr[snd, fst] = 0
    end
    
    for i in 1:nadd
        
        edge = add_edges[i]
        fst = edge[1]
        snd = edge[2]
        flag = rand(Uniform(0,1))
        if flag < 0.5
            val = rand(Uniform(0.4, 0.6)
        else
            val = rand(Uniform(-0.6, -0.4) 
        end
        Ppr[fst, snd] = val
        Ppr[snd, fst] = val
    end
    
    #Pp, _ = prox(IndPSD(), Ppr)
        
    return(Ppr)
    
end

LoadError: syntax: missing comma or ) in argument list

In [218]:
PP2 = mutate(P1)

10×10 Array{Float64,2}:
 1.0  0.5  0.0  0.0  0.5  0.0  0.0  0.0  0.5  0.5
 0.5  1.0  0.5  0.4  0.0  0.0  0.0  0.5  0.0  0.0
 0.0  0.5  1.0  0.0  0.4  0.5  0.0  0.0  0.0  0.0
 0.0  0.4  0.0  1.0  0.5  0.4  0.0  0.0  0.0  0.0
 0.5  0.0  0.4  0.5  1.0  0.5  0.4  0.0  0.0  0.0
 0.0  0.0  0.5  0.4  0.5  1.0  0.0  0.4  0.0  0.0
 0.0  0.0  0.0  0.0  0.4  0.0  1.0  0.5  0.0  0.0
 0.0  0.5  0.0  0.0  0.0  0.4  0.5  1.0  0.5  0.4
 0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.5  1.0  0.0
 0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.4  0.0  1.0

In [49]:
P2, _ = prox(IndPSD(), PP2)

([1.015742717586749 0.4896014950919693 … -0.0009101657680611608 0.015780316661891003; 0.4896014950919693 1.006868503085727 … 0.0006011899250656927 -0.010423340147895616; … ; -0.0009101657680611608 0.0006011899250656927 … 1.0000526212657257 -0.0009123395599062167; 0.015780316661891003 -0.010423340147895616 … -0.0009123395599062167 1.0158180055366755], 0.0)

In [149]:
P2

10×10 Array{Float64,2}:
  1.01574       0.489601     0.39377      …  -0.000910166   0.0157803
  0.489601      1.00687      0.00411484       0.00060119   -0.0104233
  0.39377       0.00411484   1.00247          0.000360166   0.493755
  0.00561704   -0.00371021  -0.00222274      -0.000324749   0.00563045
  0.48136       0.0123119    0.407376         0.00107764    0.481316
  0.0081256    -0.00536719  -0.00321542   …   0.49953       0.00814501
  0.00672957    0.495555    -0.00266299       0.399611      0.00674564
 -0.0115423     0.00762401   0.00456745       0.500667      0.38843
 -0.000910166   0.00060119   0.000360166      1.00005      -0.00091234
  0.0157803    -0.0104233    0.493755        -0.00091234    1.01582

In [67]:
cholesky(P2)

Cholesky{Float64,Array{Float64,2}}
U factor:
10×10 UpperTriangular{Float64,Array{Float64,2}}:
 1.00784  0.485793   0.390707   0.00557334  …  -0.000903085   0.0156576
  ⋅       0.877994  -0.211491  -0.0073095       0.00118441   -0.0205351
  ⋅        ⋅         0.897265  -0.006627        0.00107382    0.538631
  ⋅        ⋅          ⋅         1.00094        -0.000303658   0.0089542
  ⋅        ⋅          ⋅          ⋅              0.00259453    0.538032
  ⋅        ⋅          ⋅          ⋅          …   0.613917     -0.271502
  ⋅        ⋅          ⋅          ⋅              0.106919      0.0287139
  ⋅        ⋅          ⋅          ⋅              0.213814      0.599207
  ⋅        ⋅          ⋅          ⋅              0.75233       0.0433923
  ⋅        ⋅          ⋅          ⋅               ⋅            2.58096e-8

In [51]:
distr2 = MvNormalCanon(P2)

ZeroMeanFullNormalCanon(
μ: 10-element Zeros{Float64}
h: 10-element Zeros{Float64}
J: [1.015742717586749 0.4896014950919693 … -0.0009101657680611608 0.015780316661891003; 0.4896014950919693 1.006868503085727 … 0.0006011899250656927 -0.010423340147895616; … ; -0.0009101657680611608 0.0006011899250656927 … 1.0000526212657257 -0.0009123395599062167; 0.015780316661891003 -0.010423340147895616 … -0.0009123395599062167 1.0158180055366755]
)


In [152]:
rand(distr2)

10-element Array{Float64,1}:
     -1.678841945881668e7
      1.1089220083951842e7
      6.643422869134933e6
     -5.990145172196811e6
      1.987762203043865e7
     -8.665341248755129e6
     -7.176577915005484e6
      1.2308985196284806e7
 970622.4194370332
     -1.6828515447581347e7

In [80]:
PP3 = mutate(P1)

10×10 Array{Float64,2}:
 1.0  0.0  0.4  0.0  0.5  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5
 0.4  0.0  1.0  0.5  0.4  0.0  0.5  0.5  0.0  0.0
 0.0  0.0  0.5  1.0  0.5  0.4  0.5  0.0  0.0  0.0
 0.5  0.0  0.4  0.5  1.0  0.5  0.4  0.0  0.0  0.0
 0.0  0.0  0.0  0.4  0.5  1.0  0.5  0.4  0.0  0.0
 0.0  0.0  0.5  0.5  0.4  0.5  1.0  0.5  0.4  0.0
 0.0  0.0  0.5  0.0  0.0  0.4  0.5  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.4  0.0  1.0  0.5
 0.0  0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.5  1.0

In [81]:
P3, _ = prox(IndPSD(), PP3)

([0.9999999999999997 1.2437967322753707e-15 … -1.1518563880486e-15 -2.42861286636753e-17; 1.2437967322753707e-15 0.9999999999999998 … 9.020562075079397e-17 0.4999999999999997; … ; -1.1518563880486e-15 9.020562075079397e-17 … 0.9999999999999987 0.4999999999999996; -2.42861286636753e-17 0.4999999999999997 … 0.4999999999999996 1.0], 0.0)

In [82]:
cholesky(P3)

Cholesky{Float64,Array{Float64,2}}
U factor:
10×10 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  1.2438e-15   0.4           1.02696e-15  …  -1.15186e-15  -2.42861e-17
  ⋅   1.0         -1.35308e-17  -5.46438e-16      9.02056e-17   0.5
  ⋅    ⋅           0.916515      0.545545         7.87379e-17   9.36906e-17
  ⋅    ⋅            ⋅            0.838082         7.1046e-16    3.06415e-16
  ⋅    ⋅            ⋅             ⋅              -2.56395e-16  -1.07976e-16
  ⋅    ⋅            ⋅             ⋅           …   1.95238e-16   6.76369e-17
  ⋅    ⋅            ⋅             ⋅               0.596251     -5.12159e-17
  ⋅    ⋅            ⋅             ⋅              -0.0523988     6.97938e-16
  ⋅    ⋅            ⋅             ⋅               0.801086      0.624153
  ⋅    ⋅            ⋅             ⋅                ⋅            0.600361

In [83]:
distr3 = MvNormalCanon(P3)

ZeroMeanFullNormalCanon(
μ: 10-element Zeros{Float64}
h: 10-element Zeros{Float64}
J: [0.9999999999999997 1.2437967322753707e-15 … -1.1518563880486e-15 -2.42861286636753e-17; 1.2437967322753707e-15 0.9999999999999998 … 9.020562075079397e-17 0.4999999999999997; … ; -1.1518563880486e-15 9.020562075079397e-17 … 0.9999999999999987 0.4999999999999996; -2.42861286636753e-17 0.4999999999999997 … 0.4999999999999996 1.0]
)


In [84]:
rand(distr3)

10-element Array{Float64,1}:
  1.251789425551664
 -1.20571566791961
  2.48876250378626
 -0.2589154894272607
 -3.7255265665605055
  2.282067249866615
  3.2250944776776076
 -3.479491720986727
 -0.6041484009508544
  1.2166797472930249

In [99]:
PP4 = mutate(P1)

10×10 Array{Float64,2}:
 1.0  0.5  0.4  0.0  0.5  0.0  0.0  0.0  0.0  0.0
 0.5  1.0  0.5  0.0  0.5  0.0  0.0  0.0  0.0  0.0
 0.4  0.5  1.0  0.5  0.4  0.0  0.0  0.5  0.5  0.0
 0.0  0.0  0.5  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.5  0.5  0.4  0.0  1.0  0.5  0.4  0.5  0.0  0.0
 0.0  0.0  0.0  0.0  0.5  1.0  0.5  0.4  0.0  0.0
 0.0  0.0  0.0  0.0  0.4  0.5  1.0  0.0  0.4  0.0
 0.0  0.0  0.5  0.0  0.5  0.4  0.0  1.0  0.5  0.0
 0.0  0.0  0.5  0.0  0.0  0.0  0.4  0.5  1.0  0.5
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5  1.0

In [100]:
P4, _ = prox(IndPSD(), PP4)

([1.000738815546924 0.5004550244325214 … -0.004556608679330498 0.0020912463183913044; 0.5004550244325214 1.0002802421186903 … -0.0028063408886943586 0.0012879644631879766; … ; -0.004556608679330498 -0.0028063408886943586 … 1.0281026607290102 0.48710234081474924; 0.0020912463183913044 0.0012879644631879766 … 0.48710234081474924 1.0059193545430802], 0.0)

In [101]:
cholesky(P4)

Cholesky{Float64,Array{Float64,2}}
U factor:
10×10 UpperTriangular{Float64,Array{Float64,2}}:
 1.00037  0.50027   0.401525  -0.000767445  …  -0.00455493   0.00209047
  ⋅       0.866031  0.346592  -0.000102654     -0.00060927   0.000279623
  ⋅        ⋅        0.849964   0.586619         0.578522     0.00446917
  ⋅        ⋅         ⋅         0.810355        -0.412956    -0.00591488
  ⋅        ⋅         ⋅          ⋅              -0.0864243   -0.0142842
  ⋅        ⋅         ⋅          ⋅           …   0.082179     0.00670399
  ⋅        ⋅         ⋅          ⋅               0.454323     0.0172184
  ⋅        ⋅         ⋅          ⋅               0.301329     0.0391317
  ⋅        ⋅         ⋅          ⋅               0.459815     1.00189
  ⋅        ⋅         ⋅          ⋅                ⋅           3.332e-8

In [102]:
distr4 = MvNormalCanon(P4)

ZeroMeanFullNormalCanon(
μ: 10-element Zeros{Float64}
h: 10-element Zeros{Float64}
J: [1.000738815546924 0.5004550244325214 … -0.004556608679330498 0.0020912463183913044; 0.5004550244325214 1.0002802421186903 … -0.0028063408886943586 0.0012879644631879766; … ; -0.004556608679330498 -0.0028063408886943586 … 1.0281026607290102 0.48710234081474924; 0.0020912463183913044 0.0012879644631879766 … 0.48710234081474924 1.0059193545430802]
)


In [103]:
rand(distr4)

10-element Array{Float64,1}:
 -1.1069179713330869e7
 -6.817326671694357e6
 -2.506246063650263e7
  1.1502366025885375e7
  5.098588286360711e7
  1.91995375611097e7
 -5.259687470473615e7
 -5.027851457057965e7
  6.8268607783659e7
 -3.1331739451369032e7

In [104]:
PP5 = mutate(P1)

10×10 Array{Float64,2}:
 1.0  0.5  0.4  0.0  0.0  0.0  0.5  0.0  0.0  0.0
 0.5  1.0  0.0  0.4  0.0  0.0  0.5  0.0  0.0  0.0
 0.4  0.0  1.0  0.5  0.4  0.0  0.0  0.0  0.0  0.0
 0.0  0.4  0.5  1.0  0.0  0.4  0.0  0.5  0.0  0.0
 0.0  0.0  0.4  0.0  1.0  0.0  0.0  0.0  0.5  0.0
 0.0  0.0  0.0  0.4  0.0  1.0  0.5  0.4  0.5  0.0
 0.5  0.5  0.0  0.0  0.0  0.5  1.0  0.5  0.4  0.0
 0.0  0.0  0.0  0.5  0.0  0.4  0.5  1.0  0.5  0.4
 0.0  0.0  0.0  0.0  0.5  0.5  0.4  0.5  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.4  0.0  1.0

In [105]:
P5, _ = prox(IndPSD(), PP5)

([1.0018324418484974 0.4933616972842111 … 0.002683945032194157 0.0024589412213621292; 0.4933616972842111 1.0240482736096326 … -0.009723004094695176 -0.008907893148753662; … ; 0.002683945032194157 -0.009723004094695176 … 1.0039311266230597 0.0036015675372945977; 0.0024589412213621292 -0.008907893148753662 … 0.0036015675372945977 1.0032996364578046], 0.0)

In [106]:
cholesky(P5)

Cholesky{Float64,Array{Float64,2}}
U factor:
10×10 UpperTriangular{Float64,Array{Float64,2}}:
 1.00092  0.49291    0.394909  0.00901597  …   0.00268149   0.00245669
  ⋅       0.883792  -0.200863  0.410577       -0.012497    -0.0114493
  ⋅        ⋅         0.903276  0.615114       -0.0116203   -0.0106461
  ⋅        ⋅          ⋅        0.705281        0.0361164    0.0330887
  ⋅        ⋅          ⋅         ⋅              0.638691     0.0219746
  ⋅        ⋅          ⋅         ⋅          …   0.396108    -0.0325408
  ⋅        ⋅          ⋅         ⋅              0.201455     0.0337854
  ⋅        ⋅          ⋅         ⋅              0.3538       0.821902
  ⋅        ⋅          ⋅         ⋅              0.521289    -0.568989
  ⋅        ⋅          ⋅         ⋅               ⋅           5.5755e-8

In [107]:
distr5 = MvNormalCanon(P5)

ZeroMeanFullNormalCanon(
μ: 10-element Zeros{Float64}
h: 10-element Zeros{Float64}
J: [1.0018324418484974 0.4933616972842111 … 0.002683945032194157 0.0024589412213621292; 0.4933616972842111 1.0240482736096326 … -0.009723004094695176 -0.008907893148753662; … ; 0.002683945032194157 -0.009723004094695176 … 1.0039311266230597 0.0036015675372945977; 0.0024589412213621292 -0.008907893148753662 … 0.0036015675372945977 1.0032996364578046]
)


In [136]:
rand(distr5)

10-element Array{Float64,1}:
 -1.1243111971162593e7
  4.072990221695834e7
  2.9018296847285066e7
 -5.536893321356943e7
 -2.9348551140358797e6
  2.791540520883272e7
 -3.809731058796427e7
  4.335527667789033e7
 -1.6467585387258135e7
 -1.5087055093477797e7

In [137]:
rand(distr1)

LoadError: UndefVarError: distr1 not defined

In [109]:
function partial_correlation(P, n_nodes)
    partial_corr = zeros(n_nodes,n_nodes)
    for i in 1:n_nodes
        for j in i:n_nodes
            if i == j
                partial_corr[i,j] = 1
            else
                corr = - P[i,j] / sqrt(P[i,i]*P[j,j])
                partial_corr[i,j] = corr
                partial_corr[j,i] = corr
            end
        end
    end
    return(partial_corr)
end

partial_correlation (generic function with 1 method)

In [110]:
T = 100

100

In [127]:
y1 = zeros(T, n_nodes)
changepoint = [25, 50, 75]
for i in 1:T
    if i < changepoint[1]
        y1[i,1:n_nodes] = rand(distr2)
    elseif i < changepoint[2]
        y1[i,1:n_nodes] = rand(distr3)
    elseif i < changepoint[3]
        y1[i,1:n_nodes] = rand(distr4)
    else
        y1[i,1:n_nodes] = rand(distr5)
    end
end

In [133]:
y1 = zeros(T, n_nodes)
changepoint = [25, 50, 75]
for i in 1:T
    if i < 50
        y1[i,1:n_nodes] = rand(distr2)
    else
        y1[i,1:n_nodes] = rand(distr3)
    end
end

In [288]:
hyper = hyper_pars(10, 100, 2.0, 2.0, 3.0, 1.0/3.0, 0.5)
#(p = 10, Tau = 100, w = 1, z = 1, delta = 3, tau = 1/3, theta =0.5 )
parPF = PF_pars(100, 0.7, 0.001, 3, 10, 1000, false)
#(N, ESS0, TOL, nE, nM, num_it, Laplace)
#changed N from 10 to 1, again to 1000
parMCMC = MCMC_pars(100, 0.5)
# (M, pB)

MCMC_pars(100, 0.5)

In [226]:
function make_posdef(M, n_nodes)
    
    Mcopy = copy(M)
    
    for i in 1:n_nodes #rows
        row_sum = sum(M[i,:]) - M[i,i]
        for j in 1:n_nodes
            if i != j
                Mcopy[i,j] = Mcopy[i,j] / row_sum
            end
        end
    end
            
    return(Mcopy)
    
end

make_posdef (generic function with 1 method)

In [240]:
P1

10×10 Array{Float64,2}:
 1.0  0.5  0.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.5  1.0  0.5  0.4  0.0  0.0  0.0  0.0  0.0  0.0
 0.4  0.5  1.0  0.5  0.4  0.0  0.0  0.0  0.0  0.0
 0.0  0.4  0.5  1.0  0.5  0.4  0.0  0.0  0.0  0.0
 0.0  0.0  0.4  0.5  1.0  0.5  0.4  0.0  0.0  0.0
 0.0  0.0  0.0  0.4  0.5  1.0  0.5  0.4  0.0  0.0
 0.0  0.0  0.0  0.0  0.4  0.5  1.0  0.5  0.4  0.0
 0.0  0.0  0.0  0.0  0.0  0.4  0.5  1.0  0.5  0.4
 0.0  0.0  0.0  0.0  0.0  0.0  0.4  0.5  1.0  0.5
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.4  0.5  1.0

In [247]:
PP3 = mutate(P1)
cholesky(make_posdef(PP3, 10))

LoadError: PosDefException: matrix is not Hermitian; Cholesky factorization failed.

# Run the SMC

# Simulation with no changepoints, graph Id

In [309]:
distr0 = MvNormalCanon(Id)

ZeroMeanFullNormalCanon(
μ: 10-element Zeros{Float64}
h: 10-element Zeros{Float64}
J: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)


In [314]:
hyper = hyper_pars(10, 100, 2.0, 2.0, 3.0, 1.0/3.0, 0.5)
#(p = 10, Tau = 100, w = 1, z = 1, delta = 3, tau = 1/3, theta =0.5 )
parPF = PF_pars(100, 0.7, 0.001, 3, 10, 100, false)
#(N, ESS0, TOL, nE, nM, num_it, Laplace)
#changed N from 10 to 1, again to 1000
parMCMC = MCMC_pars(100, 0.5)
# (M, pB)

MCMC_pars(100, 0.5)

In [311]:
T = 100
n_nodes = 10
y0 = zeros(T, n_nodes)
changepoint = []
for i in 1:T
    y0[i,1:n_nodes] = rand(distr0)
end

In [315]:
pair = make_pairs(hyper)
Spr = []
#temp = test_logl_est(Spr, hyper, y0, parPF, pair)

Any[]

In [316]:
(logpYS, graphs) = smc(Spr, hyper, y0, parPF, pair)

l = 1
phi = 0.05
logw = [-73.2581385997084, -72.20941166609028, -72.39496815279176, -75.3821797996831, -75.58635571054349, -69.71662753673142, -74.60406867097957, -71.0080798633529, -73.35524083069727, -74.42765255512099, -75.13475508157305, -72.30188087826396, -74.66180583655493, -67.34515341432231, -67.44800440677757, -73.87226445148733, -74.80003834610147, -72.89029412304042, -73.94553369766362, -64.50580352096787, -72.78256611402756, -75.38603223091432, -74.88439567612824, -63.92888260918525, -74.3167757465893, -75.0443426500802, -75.1229404988404, -70.70543074758719, -71.8900846039625, -63.246791479196474, -68.63477134040163, -75.28217291899499, -75.8418530941318, -73.0815887679211, -75.73300341340419, -75.07406480848165, -69.1277392555781, -68.63818600302285, -71.79476297691944, -72.0840928834878, -66.35045148876978, -73.56697218988332, -71.91418357762603, -73.46593142771418, -74.70434594292236, -75.56646103482208, -74.49723505107549, -74.86546233765444, -66.62112733159438, -74.3

LoadError: InterruptException:

In [268]:
G0 = [0 0 0 1 1 1 0 1 0 1; 0 0 0 1 1 0 1 0 0 1; 0 0 0 0 1 1 0 1 0 1; 0 0 0 0 0 1 1 0 0 1; 0 0 0 0 0 0 0 1 1 1; 0 0 0 0 0 0 1 1 1 0; 0 0 0 0 0 0 0 1 1 1; 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0]
G = zeros(1,10,10)
G[1,:,:] = G0

10×10 Array{Int64,2}:
 0  0  0  1  1  1  0  1  0  1
 0  0  0  1  1  0  1  0  0  1
 0  0  0  0  1  1  0  1  0  1
 0  0  0  0  0  1  1  0  0  1
 0  0  0  0  0  0  0  1  1  1
 0  0  0  0  0  0  1  1  1  0
 0  0  0  0  0  0  0  1  1  1
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0

In [ ]:
hyper = hyper_pars(10, 100, 2.0, 2.0, 3.0, 1.0/3.0, 0.5hyper = hyper_pars(10, 100, 2.0, 2.0, 3.0, 1.0/3.0, 0.5
#(p = 10, Tau = 100, w = 1, z = 1, delta = 3, tau = 1/3, theta =0.5 )
parPF = PF_pars(100, 0.7, 0.001, 3, 10, 00, false)
#(N, ESS0, TOL, nE, nM, num_it, Laplace)
#changed N from 10 to 1, again to 1000
parMCMCvbvbbvb vbv 
#(p = 10, Tau = 100, w = 1, z = 1, delta = 3, tau = 1/3, theta =0.5 )
parPF = PF_pars(100, 0.7, 0.001, 3, 10, 00, false)
#(N, ESS0, TOL, nE, nM, num_it, Laplace)
#changed N from 10 to 1, again to 1000
parMCMC

In [287]:
logp_GY(G, hyper, parPF, y0)

1-element Array{Float64,1}:
 -1342.6423081996022

In [267]:
graphs[1,:,:]

10×10 Array{Float64,2}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN

# Simulation with 1 changepoint, graph Id and P1

In [317]:
distr0 = MvNormalCanon(Id)

ZeroMeanFullNormalCanon(
μ: 10-element Zeros{Float64}
h: 10-element Zeros{Float64}
J: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)


In [318]:
y1 = zeros(T, n_nodes)
changepoint = [50]
for i in 1:T
    if i < changepoint[1]
        y1[i,1:n_nodes] = rand(distr0)
    else
        y1[i,1:n_nodes] = rand(distr1)
    end
end

In [319]:
distr1

FullNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.5 … 0.0 0.0; 0.5 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.5; 0.0 0.0 … 0.5 1.0]
)


In [323]:
hyper = hyper_pars(10, 100, 2.0, 2.0, 3.0, 1.0/3.0, 0.5)
#(p = 10, Tau = 100, w = 1, z = 1, delta = 3, tau = 1/3, theta =0.5 )
parPF = PF_pars(100, 0.7, 0.001, 3, 10, 500, false)
#(N, ESS0, TOL, nE, nM, num_it, Laplace)
#changed N from 10 to 1, again to 1000
parMCMC = MCMC_pars(100, 0.5)
# (M, pB

MCMC_pars(100, 0.5)

In [324]:
pair = make_pairs(hyper)
#Spr = [25,50,75]
Spr = [50]
#temp = test_logl_est(Spr, hyper, y1, parPF, pair)

1-element Array{Int64,1}:
 50

In [325]:
(logpYS, graphs) = smc(Spr, hyper, y1, parPF, pair)

l = 1
phi = 0.05
logw = [-34.49058191297945, -32.37983472675568, -37.18391886780809, -32.77410808965245, -37.56717140765095, -30.106288803890983, -26.83899665366156, -36.723247923962084, -39.16762763406925, -39.524197530618004, -39.82429071605082, -38.947489889254534, -39.5439367278778, -38.486188299371335, -34.08017785291478, -40.02458047882629, -39.10223003076973, -40.48297074720844, -36.592722389936625, -38.028369440699464, -39.19319138800784, -39.359393454138676, -36.49869628907778, -35.7245072288235, -40.6703994723399, -34.700120046149905, -38.54327991239803, -38.51210675885887, -39.576951795949334, -38.63484628679615, -36.69638457331035, -39.25393872283221, -37.62693061236556, -36.03870384284627, -32.12095091877465, -35.526972863337896, -40.572484863956085, -31.281602765582335, -37.18974422663569, -31.002143996492848, -36.95879763333412, -36.53775257835063, -40.18912247745828, -30.383817664128845, -36.798419001203996, -31.90903233176964, -34.97162131864657, -35.2407895802587, -37

phi = 0.25
logw = [-21.642352513752154, -26.3097760033258, -21.32250135036436, -23.68104474836874, -26.5094797327079, -23.008679943308262, -21.016474446188376, -17.341028714083276, -24.802455550584632, -30.321603377677608, -20.432536020088218, -29.147767240809944, -25.24675507664002, -21.495283588889528, -21.274685124092798, -26.455294633298372, -20.148602544406028, -25.142963861642873, -23.631940303344074, -25.826257652370856, -22.851037610996976, -19.480116428160724, -25.44324500043343, -23.77443705514518, -23.782722190201515, -29.191283162346423, -27.113886029867853, -19.162824164272546, -24.489313833503996, -21.58852803297954, -21.53434652794626, -19.091662404937296, -27.668490324388387, -19.86731999342561, -25.643027897994035, -27.47423813252836, -22.387260243133984, -24.37946393030104, -27.16970606727377, -28.28222808193735, -18.28162669390089, -19.89574395525621, -25.054756419605617, -23.011335367070092, -24.4152384210001, -22.600003463269342, -25.263697875118247, -24.0928222399

phi = 0.44999999999999996
logw = [-25.99318271111801, -27.711404201536492, -23.403839044573488, -30.040509457877324, -23.8870919806872, -22.714248335362075, -23.791828976739307, -23.37278055705526, -24.62150601294597, -24.208764020972502, -26.235324589723643, -19.58036185035337, -27.222637877878718, -22.474316893646144, -21.592327708087556, -30.58564335561867, -20.991825932581815, -25.687361260494434, -23.02638981493443, -24.696774527763957, -24.488224097490836, -26.7265317795836, -28.208846086015313, -24.87294171511578, -27.57821941497244, -28.793646695485524, -19.22517881700139, -23.99516949334325, -24.771558405802296, -23.795563137598254, -22.252987194869196, -25.85431663305213, -25.85179304969777, -29.53917350932746, -22.893102152583037, -22.184998246630197, -19.242116925937143, -24.646332126058326, -22.918319740858447, -22.391355935709935, -24.8473613469489, -27.238344428561515, -24.100424622725964, -20.342819631337235, -34.91837819697614, -24.472288580140095, -27.7614849529632, -

phi = 0.65
logw = [-17.880304689504737, -17.505146036396056, -11.115943243208509, -20.435525703608363, -18.215333776646485, -11.073418317733658, -12.916345898320008, -18.456937863578656, -17.439649964680818, -16.028099040587357, -16.455834003106734, -11.697117682605642, -11.646262534424453, -9.984004495482683, -11.011572488435222, -17.30141558755358, -14.42487705351423, -20.088503412697033, -25.59248284244137, -17.00126674951371, -16.10162879012126, -11.656001962453939, -19.52779835833208, -18.132908860842644, -16.000306627823633, -16.1819527134403, -10.444451126457139, -13.689025530901052, -15.929305867473241, -11.199565251467606, -9.976566751520275, -18.960800781543412, -11.423994329336544, -11.726290633967414, -18.72362884361745, -18.016141088492894, -11.390998174218844, -11.576022344811445, -16.707064806723704, -15.416111032767285, -18.06761376546442, -17.611253860505165, -16.745271684726603, -18.47125830415984, -10.9799736021508, -18.600841730930256, -10.916490507136226, -15.82656

phi = 0.8500000000000002
logw = [-11.891438104674625, -11.27066815416025, -10.342996283719371, -10.040215478720043, -14.942405734734713, -11.380550320898553, -11.390048659209082, -11.917366751438083, -14.611021752276818, -10.501835314836539, -10.323447446168544, -11.264056040649784, -11.796535307220761, -11.742305896145472, -9.703667954037522, -10.667684277436608, -9.66160203208307, -11.149973772560859, -10.997233673418025, -9.900465136193171, -10.693982586115604, -11.075368029264101, -11.560520678208222, -11.502481218026224, -16.178891499446483, -11.16688364140246, -11.438096461883417, -10.333988690344448, -10.160391292969658, -10.335733220102234, -11.562353202775386, -11.320227200987585, -16.575494784569084, -11.501772352469038, -16.39215435779821, -9.73647311654473, -11.970983286889002, -11.729223646958319, -11.056425443253946, -10.396664517722625, -15.78929061202796, -16.967992360132005, -9.99882436404918, -12.017325863040355, -11.502705005731586, -11.147114021512866, -10.826107237

...

phi = 0.25
logw = [-30.562374886122612, -29.64805271875404, -31.984426118108882, -31.35596151837127, -29.09396129907256, -31.532739159285022, -33.65774802785749, -32.659361169231694, -32.026934228175385, -33.41873395893377, -31.35887598382539, -32.163836749919746, -29.89310435618459, -32.423574810508526, -28.04067816636888, -28.6534035887755, -29.76418444118163, -28.723573314058257, -33.333639752049166, -34.419079693849646, -29.999833720011956, -34.19824109883303, -35.06329302640693, -28.54006418734048, -33.393253857652844, -33.58449188663644, -30.25433795023917, -29.925866067593635, -29.006538681402976, -30.67092045163517, -32.02916674261685, -33.35462991303982, -29.292938092698652, -30.940539626025757, -33.05250185531842, -29.922992923444685, -30.01186859877256, -28.291943587787816, -30.19871295554589, -30.0300308576482, -34.49502251137954, -30.509356161349732, -32.40348850991803, -30.486302018794063, -28.205045677889704, -30.60363945919314, -32.54145762476461, -29.92559775454717, -2

phi = 0.44999999999999996
logw = [-31.455569578627948, -29.081459214164777, -30.484620023238122, -33.08234246831686, -38.877854493575924, -29.903823979036968, -32.721292856464544, -33.174905729915324, -30.299636746159642, -35.83284477940722, -32.703903234165665, -32.761254219446734, -32.77100256762875, -29.496928112920017, -30.419892658608756, -31.344995621375123, -31.194006797960917, -34.28109232705355, -35.54145002336435, -32.42331123721716, -32.272986132161954, -29.45348719114641, -27.550959235095494, -31.98741068335041, -31.81469910542964, -33.78235655758184, -31.71637480726013, -34.839866038635854, -38.52791346230592, -31.55659398746178, -26.50118053322553, -34.868931290102175, -33.243701996435064, -28.197234556126574, -32.82387579518355, -33.01685294784823, -30.192699607198033, -28.70000085954212, -31.36219779148784, -30.170988060032094, -35.82744705151308, -30.917815730583747, -29.958975243331697, -32.646173423301335, -28.032616149284564, -29.546489920517278, -29.439963219439875

phi = 0.65
logw = [-27.512851328035346, -36.84652322117737, -32.087726300949086, -29.676868865249688, -32.5423063379621, -32.10663652505475, -33.56375405288268, -29.43866899220725, -26.96527205361199, -30.849522739338152, -33.639165061993644, -34.700930128865124, -35.07930924813388, -30.57804099672242, -32.29491647005405, -30.36756398395818, -33.47489894868877, -29.66693359985785, -29.234210796870652, -28.916971939636486, -32.56478285475211, -38.51068264908447, -33.14546988384935, -28.249310888478522, -27.48716178853315, -29.626548046296094, -31.99622999016097, -29.541492360772892, -28.793470566329173, -31.13420172870528, -29.181325352246965, -29.173137824397973, -29.364008527323843, -35.86233842775614, -31.05143088833533, -32.36868253921818, -30.880177448047892, -31.96143869336483, -31.550040936909795, -31.44593951968412, -33.48496670572242, -28.36752950073883, -25.1836358784903, -31.71052208636623, -32.66393454764149, -33.118250076229316, -33.26728752997348, -31.888051930073477, -31.

phi = 0.8500000000000002
logw = [-34.775886820918316, -30.98614827458843, -27.891610843081626, -33.05273508009757, -31.60601663116975, -28.554323179520626, -28.986231339372555, -28.870681839332523, -25.077143415710598, -31.76647035988912, -34.055201407340284, -32.170007231673836, -27.13957507073318, -29.178019357522682, -29.241607154374176, -34.20931384285826, -32.48431154084272, -29.03428851108513, -30.738806395869815, -28.24332424012763, -31.14956803149721, -31.87849502051111, -28.902658630278715, -30.641474410651078, -23.67563018491741, -30.71393344564629, -32.367267819031866, -27.648817217675557, -30.340678522935153, -26.414530565707025, -30.969122200215775, -33.46231469280172, -31.51719595619274, -29.24081716842499, -33.06700861910039, -29.92441540138603, -30.582014684534784, -30.497732625574727, -30.338478825938637, -29.998283173770485, -36.29345046829277, -33.24389937402887, -31.105512452873512, -29.769138285026454, -31.92630587019725, -31.893742546096775, -26.933951288558873, -

(-898.1791999978116, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 5.810463023472858e-24 4.598858116507817e-19 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0; 1.3088989764496507e-19 1.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0; 1.0 1.994161826008071e-34 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[1.0 1.0 … 0.0 0.0; 1.0 9.519170496424432e-31 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0; 6.203622135409484e-34 1.0 … 4.598802115809655e-19 0.0])

In [326]:
#graph for y[1:49]
graphs[1,:,:]

10×10 Array{Float64,2}:
 0.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0
 0.0  0.0  1.0  0.0  1.0  0.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [327]:
#graph for y[50:100]
graphs[2,:,:]

10×10 Array{Float64,2}:
 0.0  1.0  0.0  5.81046e-24  1.3089e-19   …  1.0          6.20362e-34
 0.0  0.0  1.0  4.59886e-19  1.0             0.0          1.0
 0.0  0.0  0.0  1.0          1.0             5.60007e-24  0.0
 0.0  0.0  0.0  0.0          6.20362e-34     5.60007e-24  1.0
 0.0  0.0  0.0  0.0          0.0             1.0          5.60007e-24
 0.0  0.0  0.0  0.0          0.0          …  1.0          5.60007e-24
 0.0  0.0  0.0  0.0          0.0             1.0          1.0
 0.0  0.0  0.0  0.0          0.0             1.0          1.0
 0.0  0.0  0.0  0.0          0.0             0.0          4.5988e-19
 0.0  0.0  0.0  0.0          0.0             0.0          0.0

In [301]:
hyper = hyper_pars(10, 100, 2.0, 2.0, 3.0, 1.0/3.0, 0.5)
#(p = 10, Tau = 100, w = 1, z = 1, delta = 3, tau = 1/3, theta =0.5 )
parPF = PF_pars(100, 0.7, 0.001, 3, 10, 100, false)
#(N, ESS0, TOL, nE, nM, num_it, Laplace)
#changed N from 10 to 1, again to 1000
parMCMC = MCMC_pars(100, 0.5)
# (M, pB)

MCMC_pars(100, 0.5)

In [302]:
Spr = [50]
pair = make_pairs(hyper)
temp = test_logl_est(Spr, hyper, y1, parPF, pair)

LoadError: UndefVarError: S not defined

In [305]:
(logpYS, graphs) = logl_est(Spr, hyper, y1, parPF, temp, pair)

w_std./sum(w_std) = [0.0056034473509086, 0.009114760092269818, 0.008134436498037311, 0.005752174518946799, 0.009277875287369203, 0.006476434385962323, 0.005932255702862307, 0.005857221466053156, 0.004488076400446329, 0.011254202748022668, 0.005894466666921129, 0.009064013839001012, 0.011706591749398455, 0.005661115643143404, 0.021168210001277003, 0.004897292438732447, 0.006764802600030514, 0.006795837807311624, 0.02981026708112581, 0.006340792016256812, 0.0050066401605955105, 0.006654361020537563, 0.007807103738441386, 0.010292304009622026, 0.013683557675083427, 0.009432845935118029, 0.03623617099261128, 0.009446111186895206, 0.012084321905348784, 0.015541494074380505, 0.00973579595603733, 0.0065331037660844435, 0.00569213123264041, 0.007050963442158832, 0.005644917854868617, 0.012917373485890407, 0.007928157757729518, 0.0060299283575825455, 0.010724905313262456, 0.007185680997068382, 0.007899603724752902, 0.006554807526842969, 0.005834960023100212, 0.008265642865198182, 0.007811593931

w_std./sum(w_std) = [0.0067060502629005606, 0.03463921778685704, 0.01833489418481659, 0.004401371447756297, 0.015183106588021475, 0.009963123959203615, 0.006934840242877548, 0.007820211049603502, 0.014929389424979846, 0.006873399308830145, 0.003594421914868725, 0.0059506982057013925, 0.006978976490732406, 0.010303777751343312, 0.021507353090257374, 0.014207591779319807, 0.014231233628617602, 0.013102606464368649, 0.005651391961252411, 0.010163953870999424, 0.009223588042230077, 0.006792358013682451, 0.010605508762250712, 0.007629143134577862, 0.007350359603526175, 0.0075633315897517955, 0.008760223469276707, 0.0034230973434745317, 0.013093740485287072, 0.008886870473558944, 0.026814011355979595, 0.0062839292995071116, 0.006063950339411848, 0.004043669472885472, 0.015135976836761326, 0.002288192544582816, 0.006194895531094813, 0.009450182279563967, 0.0035305743081150208, 0.010530636828178794, 0.002617208135764973, 0.01300513687348988, 0.002360956763505536, 0.008566470259926186, 0.004651

w_std./sum(w_std) = [0.014507028530815393, 0.009648973855084408, 0.003165700970898636, 0.01599871768019688, 0.021149746609684685, 0.008493660381672052, 0.006084121971800405, 0.013701684795248623, 0.0060801494076052915, 0.003742536125503297, 0.0032170894113053706, 0.007536954352491845, 0.012465597799327334, 0.004938821136127287, 0.004423651781351781, 0.008832366342244087, 0.003451381592620021, 0.0039027510524776107, 0.01810314532708936, 0.010421137641136407, 0.01189520674615438, 0.00679791716817865, 0.008049149159736204, 0.016491286139150865, 0.010200345493456102, 0.007133096489466407, 0.006383737299274693, 0.009615700048101406, 0.00879898549479363, 0.03500000820544794, 0.010396262772148198, 0.002342038987039575, 0.014932549697456835, 0.022250741476426146, 0.003863317691196607, 0.01187237219074258, 0.012065076088661631, 0.019183721456736996, 0.009462619236316016, 0.004762701279932963, 0.010701336279646017, 0.009354212933968464, 0.005904367318564169, 0.007646622055901366, 0.0140876607378

LoadError: ArgumentError: Categorical: the condition isprobvec(p) is not satisfied.

In [306]:
graphs[1,:,:]

10×10 Array{Float64,2}:
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [307]:
graphs[2,:,:]

10×10 Array{Float64,2}:
 0.0  0.0  1.0  2.80967e-38  0.000206083  …  0.0          1.0
 0.0  0.0  1.0  1.0          1.0             0.0          1.0
 0.0  0.0  0.0  0.999794     1.0             1.0          0.0
 0.0  0.0  0.0  0.0          1.0             1.51964e-15  0.000206083
 0.0  0.0  0.0  0.0          0.0             0.0          4.6556e-11
 0.0  0.0  0.0  0.0          0.0          …  0.000206083  0.999794
 0.0  0.0  0.0  0.0          0.0             0.999794     0.0
 0.0  0.0  0.0  0.0          0.0             1.0          0.0
 0.0  0.0  0.0  0.0          0.0             0.0          1.0
 0.0  0.0  0.0  0.0          0.0             0.0          0.0

In [89]:
partial_correlation(PP2, 10)

10×10 Array{Float64,2}:
  1.0        -0.418509   -0.334875   -0.0352824   …  -0.464064    -0.0184129
 -0.418509    1.0        -0.0569568  -0.337838       -0.0270574    0.0225549
 -0.334875   -0.0569568   1.0        -0.443369       -0.0233262   -0.466245
 -0.0352824  -0.337838   -0.443369    1.0             0.0177001   -0.0147547
 -0.0401222  -0.425138   -0.340489   -0.0321509       0.0201281   -0.0167787
  0.0236056  -0.0289157  -0.0249282   0.0189158   …  -0.0118422    0.00987159
 -0.0291415  -0.442821    0.0307743  -0.0233518      -0.379304    -0.0121866
  0.011259   -0.0137917  -0.0118898   0.00902211     -0.502154     0.00470838
 -0.464064   -0.0270574  -0.0233262   0.0177001       1.0          0.00923718
 -0.0184129   0.0225549  -0.466245   -0.0147547       0.00923718   1.0

In [93]:
graphs[2,:,:]

10×10 Array{Float64,2}:
 0.0  1.0  1.0         0.999996    …  0.999996    0.999996    1.69045e-7
 0.0  0.0  4.27198e-6  1.69294e-7     0.999996    1.0         4.24362e-6
 0.0  0.0  0.0         4.34244e-6     0.999996    1.0         1.0
 0.0  0.0  0.0         0.0            1.69294e-7  0.999996    0.999996
 0.0  0.0  0.0         0.0            0.999996    1.0         9.85467e-8
 0.0  0.0  0.0         0.0         …  1.0         7.07662e-8  1.0
 0.0  0.0  0.0         0.0            0.999996    4.34244e-6  8.23178e-13
 0.0  0.0  0.0         0.0            0.0         4.17372e-6  1.69295e-7
 0.0  0.0  0.0         0.0            0.0         0.0         1.0
 0.0  0.0  0.0         0.0            0.0         0.0         0.0

In [94]:
PP3

10×10 Array{Float64,2}:
  1.0307       0.475397     0.408893    …   0.0206837    0.475884
  0.475397     1.01972     -0.0071263      -0.0165742    0.0193246
  0.408893    -0.0071263    1.00258         0.00599096  -0.00698516
  0.0338592    0.372868     0.00980721      0.0228094   -0.0265945
  0.469485     0.0244524    0.391161       -0.0205567    0.0239681
 -0.0210402    0.0168598    0.493906    …   0.485826     0.0165259
 -0.012502     0.010018    -0.00362116      0.391578     0.00981962
  0.00657384  -0.00526773   0.00190409      0.504428     0.394837
  0.0206837   -0.0165742    0.00599096      1.01393      0.483754
  0.475884     0.0193246   -0.00698516      0.483754     1.01894

In [ ]:
#try 100 data points from one graph
#also precision matrices can't have negatuve entries